# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.



In [1]:
# Your code goes here
# Dataset URL = https://bit.ly/3BPcobU
#
import pandas as pd

#read the data into chunk of 5000 rows
crunch_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
crunch_sample = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", nrows=10)
print(f'A sample of the data:\n{crunch_sample}')

#undestand each chunk missing values, memory footprint, and total memory
chunk_number = 0
chunk_memory_usage = 0
total_memory_usage = 0
missing_value = []
for chunk in crunch_iter:
  #missing value for each chunk column
  cols_missing_value = chunk.isnull().sum()
  print(f'\nchunk:{chunk_number}: columns missing value count\n{cols_missing_value}')
  print(f'\nchunk:{chunk_number}: column total count\n{chunk.shape[0]}')
  chunk_number +=1

  #chunks memory usage
  chunk_memory_usage = chunk.memory_usage(deep=True).sum()/(1024*1024)
  total_memory_usage +=chunk_memory_usage
  print(f'chunk:{chunk_number} memory usage is:{chunk_memory_usage}')
  
  #determine which columns to drop based on missing values
  #calculate the percentage of missing value to total value count per colum
  missing_value.append( 100*(cols_missing_value/chunk.shape[0]) )
  print(f'\nchunk:{chunk_number}: column total count\n{100*(cols_missing_value/chunk.shape[0]) }')

#concatenate the list of series with sum of missing value per chunk column into one series
total_missing_value = pd.concat(missing_value)
#group up the columns missing values
total_missing_value = total_missing_value.groupby(total_missing_value.index).mean()
#columns with alot of missing value that can be dropped i.e with 
most_na_cols_to_drop = list(total_missing_value[total_missing_value > 95].index)
few_na_cols_to_drop = list(total_missing_value[(total_missing_value > 25) & (total_missing_value <95) ].index)

print(f'\nmost null value columns to drop columns:{most_na_cols_to_drop}')
print(f'\nfew null value columns to drop rows:{few_na_cols_to_drop}')
print(f'\ntotal memory footprint is:{total_memory_usage}')
print(f'\npercentage of missing value per column\n:{total_missing_value}')



A sample of the data:
     company_permalink company_name company_category_code  \
0    /company/advercar     AdverCar           advertising   
1  /company/launchgram   LaunchGram                  news   
2        /company/utap         uTaP             messaging   
3    /company/zoopshop     ZoopShop              software   
4    /company/efuneral     eFuneral                   web   
5       /company/tackk        Tackk                   web   
6    /company/acclaimd     Acclaimd             analytics   
7    /company/acclaimd     Acclaimd             analytics   
8    /company/toviefor     ToVieFor             ecommerce   
9    /company/ohk-labs     OHK Labs                sports   

  company_country_code company_state_code         company_region  \
0                  USA                 CA                 SF Bay   
1                  USA                 CA                 SF Bay   
2                  USA                NaN  United States - Other   
3                  USA            

## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [2]:
#iterate through df columns and downcast dtype
def mem_optimisation(df, columns, convert_dtype):
  for c in columns:
    df[c] = pd.to_numeric(df[c], downcast=convert_dtype, errors='coerce')
  return df

In [3]:
# Your code goes here
#dtype for the columns
print(f'columns datatype{crunch_sample.info()}')

#read the data into chunk of 5000 rows
crunch_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
#determine the columns with the least unique values
values_uniqueness = []
for chunk in crunch_iter:
  #determine the values count per columns. columns with less unique <50 can be converted to category data type
  chunk_unique_values = chunk.select_dtypes(include=['object']).nunique() #pd.series with unique values per col
  chunk_values_count = chunk.select_dtypes(include=['object']).count() #total no of items in col
  values_uniqueness.append((100*chunk_unique_values/chunk_values_count)) #add each chunk series to a list

#concatenate the series in the list to a series
total_values_uniqueness = pd.concat(values_uniqueness)
total_values_uniqueness = total_values_uniqueness.groupby(total_values_uniqueness.index).mean()
categorise_cols = list((total_values_uniqueness[total_values_uniqueness <50]).index)

print(f'columns values uniqueness:\n{total_values_uniqueness}')
print(f'columns to categorise:\n{categorise_cols}')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   company_permalink       10 non-null     object
 1   company_name            10 non-null     object
 2   company_category_code   10 non-null     object
 3   company_country_code    10 non-null     object
 4   company_state_code      9 non-null      object
 5   company_region          10 non-null     object
 6   company_city            9 non-null      object
 7   investor_permalink      10 non-null     object
 8   investor_name           10 non-null     object
 9   investor_category_code  7 non-null      object
 10  investor_country_code   8 non-null      object
 11  investor_state_code     8 non-null      object
 12  investor_region         10 non-null     object
 13  investor_city           8 non-null      object
 14  funding_round_type      10 non-null     object
 15  funded_at

In [4]:

#read the data into chunk of 5000 rows
crunch_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
"""
optimise the memory by optimising the any int or float columns using pd.to_numeric(). 
convert less unique columns to category dtype
drop columns with very high
"""
chunk_memory_usage = 0
opti_total_memory_usage = 0
for chunk in crunch_iter:
  #drops rows where cols with null values are null
  chunk = chunk.dropna(axis=0, subset=few_na_cols_to_drop)

  #pick columns with integer dtype
  int_cols = chunk.select_dtypes(include='integer').columns
  float_cols = chunk.select_dtypes(include='float').columns
  #float_cols = ['funded_year', 'raised_amount_usd']
  #use the pd.to_numeric to optimise the space for the int
  chunk = mem_optimisation(chunk, int_cols, 'integer')
  chunk = mem_optimisation(chunk, float_cols, 'float')

  #convert columns with few unique values to category dtype
  chunk[categorise_cols] = chunk[categorise_cols].astype('category')
 
  #drop rows where there are missing values  
  chunk = chunk.drop(columns=most_na_cols_to_drop)
 

  #chunk memory usage
  chunk_memory_usage = chunk.memory_usage(deep=True).sum()/(1024*1024)
  opti_total_memory_usage +=chunk_memory_usage

print(f'\ntotal memory footprint is:{opti_total_memory_usage}')
print(f'Optimised memory is {total_memory_usage/opti_total_memory_usage} times less than original memory')




total memory footprint is:9.356792449951172
Optimised memory is 6.090646119763033 times less than original memory


In [5]:
crunch_sample = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1",)
crunch_sample.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52870 entries, 0 to 52869
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       52869 non-null  object 
 1   company_name            52869 non-null  object 
 2   company_category_code   52227 non-null  object 
 3   company_country_code    52869 non-null  object 
 4   company_state_code      52378 non-null  object 
 5   company_region          52869 non-null  object 
 6   company_city            52337 non-null  object 
 7   investor_permalink      52868 non-null  object 
 8   investor_name           52868 non-null  object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   40869 non-null  object 
 11  investor_state_code     36061 non-null  object 
 12  investor_region         52868 non-null  object 
 13  investor_city           40390 non-null  object 
 14  funding_round_type      52867 non-null

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [6]:
# Your code goes here
import sqlite3
conn = sqlite3.connect('crunch.db')

#read the data into chunk of 5000 rows
crunch_iter = pd.read_csv('https://bit.ly/3BPcobU', encoding = "ISO-8859-1", chunksize=5000)
"""
optimise the memory by optimising the any int or float columns using pd.to_numeric(). 
convert less unique columns to category dtype
drop columns with very high
"""
chunk_memory_usage = 0
opti_total_memory_usage = 0
for chunk in crunch_iter:
  #drops rows where cols with null values are null
  chunk = chunk.dropna(axis=0, subset=few_na_cols_to_drop)

  #pick columns with integer dtype
  int_cols = chunk.select_dtypes(include='integer').columns
  float_cols = chunk.select_dtypes(include='float').columns
  #float_cols = ['funded_year', 'raised_amount_usd']
  #use the pd.to_numeric to optimise the space for the int
  chunk = mem_optimisation(chunk, int_cols, 'integer')
  chunk = mem_optimisation(chunk, float_cols, 'float')

  #convert columns with few unique values to category dtype
  chunk[categorise_cols] = chunk[categorise_cols].astype('category')
 
  #drop rows where there are missing values  
  chunk = chunk.drop(columns=most_na_cols_to_drop)

  #store the data in crunch databse
  chunk.to_sql('crunch', conn, if_exists='append', index=False)

In [7]:
#querying the created table crunch from crunch database
crunch_sql_dtype = pd.read_sql('PRAGMA table_info(crunch);', conn)
print(crunch_sql_dtype)

    cid                   name     type  notnull dflt_value  pk
0     0      company_permalink     TEXT        0       None   0
1     1           company_name     TEXT        0       None   0
2     2  company_category_code     TEXT        0       None   0
3     3   company_country_code     TEXT        0       None   0
4     4     company_state_code     TEXT        0       None   0
5     5         company_region     TEXT        0       None   0
6     6           company_city     TEXT        0       None   0
7     7     investor_permalink     TEXT        0       None   0
8     8          investor_name     TEXT        0       None   0
9     9  investor_country_code     TEXT        0       None   0
10   10    investor_state_code     TEXT        0       None   0
11   11        investor_region     TEXT        0       None   0
12   12          investor_city     TEXT        0       None   0
13   13     funding_round_type     TEXT        0       None   0
14   14              funded_at     TEXT 

In [8]:
crunch_sample

,company_permalink,company_name,company_category_code,company_country_code,company_state_code,company_region,company_city,investor_permalink,investor_name,investor_category_code,investor_country_code,investor_state_code,investor_region,investor_city,funding_round_type,funded_at,funded_month,funded_quarter,funded_year,raised_amount_usd
0,/company/advercar,AdverCar,advertising,USA,CA,SF Bay,San Francisco,/company/1-800-flowers-com,1-800-FLOWERS.COM,NaN,USA,NY,New York,New York,series-a,2012-10-30,2012-10,2012-Q4,2012.0,2000000.0
1,/company/launchgram,LaunchGram,news,USA,CA,SF Bay,Mountain View,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-23,2012-01,2012-Q1,2012.0,20000.0
2,/company/utap,uTaP,messaging,USA,NaN,United States - Other,NaN,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2012-01-01,2012-01,2012-Q1,2012.0,20000.0
3,/company/zoopshop,ZoopShop,software,USA,OH,Columbus,columbus,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,angel,2012-02-15,2012-02,2012-Q1,2012.0,20000.0
4,/company/efuneral,eFuneral,web,USA,OH,Cleveland,Cleveland,/company/10xelerator,10Xelerator,finance,USA,OH,Columbus,Columbus,other,2011-09-08,2011-09,2011-Q3,2011.0,20000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52865,/company/garantia-data,Garantia Data,enterprise,USA,CA,SF Bay,Santa Clara,/person/zohar-gilon,Zohar Gilon,NaN,NaN,NaN,unknown,NaN,series-a,2012-08-08,2012-08,2012-Q3,2012.0,3800000.0
52866,/company/duda-mobile,DudaMobile,mobile,USA,CA,SF Bay,Palo Alto,/person/zohar-gilon,Zohar Gilon,NaN,NaN,NaN,unknown,NaN,series-c+,2013-04-08,2013-04,2013-Q2,2013.0,10300000.0
52867,/company/sitebrains,SiteBrains,software,USA,CA,SF Bay,San Francisco,/person/zohar-israel,zohar israel,NaN,NaN,NaN,unknown,NaN,angel,2010-08-01,2010-08,2010-Q3,2010.0,350000.0
52868,/company/comprehend-systems,Comprehend Systems,enterprise,USA,CA,SF Bay,Palo Alto,/person/zorba-lieberman,Zorba Lieberman,NaN,NaN,NaN,unknown,NaN,series-a,2013-07-11,2013-07,2013-Q3,2013.0,8400000.0


## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [9]:
# Your code goes here
#What proportion of the total amount of funds did the top 10% raise? 
#read the fund raised from the database
fund_raised = pd.read_sql('select raised_amount_usd from crunch;', conn)
total_raised = fund_raised.sum() #calculate total amount raised
#get fund raised by 10%. each row is an  unique investiment
percent_10_investiment = int((fund_raised.count())*0.1)
top_10_raised_amount = fund_raised.nlargest(percent_10_investiment, columns='raised_amount_usd', keep='all').sum()
top_10_share = 100*(top_10_raised_amount/total_raised)
#get fund raised by 1%. each row is an  unique investiment
percent_1_investiment = int((fund_raised.count())*0.01)
top_1_raised_amount = fund_raised.nlargest(percent_1_investiment, columns='raised_amount_usd', keep='all').sum()
top_1_share = 100*(top_1_raised_amount/total_raised)
#get fund raised by bottom 10%. each row is an  unique investiment
bottom_10_raised_amount = fund_raised.nsmallest(percent_10_investiment, columns='raised_amount_usd', keep='all').sum()
bottom_10_share = 100*(bottom_10_raised_amount/total_raised)
#get fund raised by 1%. each row is an  unique investiment
bottom_1_raised_amount = fund_raised.nsmallest(percent_1_investiment, columns='raised_amount_usd', keep='all').sum()
bottom_1_share = 100*(bottom_1_raised_amount/total_raised)

print(f'proportion of the total amount of funds the top 10% raised:{top_10_share}')
print(f'proportion of the total amount of funds the top 1% raised:{top_1_share}')

print(f'proportion of the total amount of funds the bottom 10% raised:{bottom_10_share}')
print(f'proportion of the total amount of funds the bottom 1% raised:{bottom_1_share}')

#Compare these values to the proportions the bottom 10% and bottom 1% raised.
print(f'top 10% share was {int(top_10_share/bottom_10_share)} times more than bottom 10% share')
print(f'top 1% share was {int(top_1_share/bottom_1_share)} times more than bottom 1% share')



proportion of the total amount of funds the top 10% raised:raised_amount_usd    47.32389
dtype: float64
proportion of the total amount of funds the top 1% raised:raised_amount_usd    18.72475
dtype: float64
proportion of the total amount of funds the bottom 10% raised:raised_amount_usd    0.399577
dtype: float64
proportion of the total amount of funds the bottom 1% raised:raised_amount_usd    0.00125
dtype: float64
top 10% share was 118 times more than bottom 10% share
top 1% share was 14980 times more than bottom 1% share


In [10]:
crunch_sample.columns

Index(['company_permalink', 'company_name', 'company_category_code',
       'company_country_code', 'company_state_code', 'company_region',
       'company_city', 'investor_permalink', 'investor_name',
       'investor_category_code', 'investor_country_code',
       'investor_state_code', 'investor_region', 'investor_city',
       'funding_round_type', 'funded_at', 'funded_month', 'funded_quarter',
       'funded_year', 'raised_amount_usd'],
      dtype='object')

In [11]:
#Which category of company attracted the most investments?
sql_query ="""
select company_name, sum(raised_amount_usd) as raised_amount 
from crunch 
where raised_amount_usd is not null 
group by company_name;
"""
company_invested_in = pd.read_sql(sql_query, conn)
print(f'the top 20 company invested in:\n{company_invested_in.nlargest(20, columns="raised_amount", keep="all")}')

the top 20 company invested in:
             company_name  raised_amount
1600            Clearwire   2.718000e+10
3140              Groupon   9.066000e+09
4780            Nanosolar   2.995000e+09
8536                Zynga   2.805639e+09
1026         Bloom Energy   2.400000e+09
4139         LivingSocial   2.394000e+09
2698    Fisker Automotive   2.358000e+09
2236              Dropbox   2.263200e+09
7144         SurveyMonkey   2.250000e+09
2594             Facebook   2.172700e+09
8149       Wave Broadband   2.100000e+09
1168  BrightSource Energy   2.024000e+09
5286  Pacific Biosciences   2.003440e+09
7229              TRUECar   1.906500e+09
6836             Solyndra   1.833000e+09
3793                KAYAK   1.813500e+09
2591              Fab.com   1.767925e+09
2112         Demand Media   1.567693e+09
2982          Gilt Groupe   1.556000e+09
4919       NextG Networks   1.540000e+09


In [12]:
#Which investor contributed the most money (across all startups)?
sql_query = """
select investor_name, sum(raised_amount_usd) as raised_amount
from crunch 
where raised_amount_usd is not null 
group by investor_name;
"""
top_investor = pd.read_sql(sql_query, conn)
print(f'the top 20 investor :\n{top_investor.nlargest(20, columns="raised_amount", keep="all")}')

the top 20 investor :
                         investor_name  raised_amount
1483  Kleiner Perkins Caufield & Byers   1.121783e+10
1873         New Enterprise Associates   9.692542e+09
52                      Accel Partners   6.472126e+09
1132                     Goldman Sachs   6.375459e+09
2420                   Sequoia Capital   6.039402e+09
1357                             Intel   5.969200e+09
1136                            Google   5.808800e+09
2761                       Time Warner   5.730000e+09
637                            Comcast   5.669000e+09
1175                 Greylock Partners   4.960983e+09
414                        BrightHouse   4.700000e+09
1358                     Intel Capital   4.695617e+09
821      Draper Fisher Jurvetson (DFJ)   4.501461e+09
1975           Oak Investment Partners   4.350065e+09
164                Andreessen Horowitz   4.233674e+09
1572       Lightspeed Venture Partners   4.140979e+09
328                          Benchmark   4.119949e+09
306   

In [13]:
#Which investors contributed the most money per startup?
sql_query = """
select investor_name, company_name, sum(raised_amount_usd) as raised_amount 
from crunch 
where raised_amount_usd is not null 
group by investor_name, company_name;
"""
top_investor_per_startup = pd.read_sql(sql_query, conn)
print(f'the top 20 investor per startup :\n{top_investor_per_startup.nlargest(20, columns="raised_amount", keep="all")}')

the top 20 investor per startup :
                          investor_name    company_name  raised_amount
4852                            Comcast       Clearwire   5.620000e+09
10965                             Intel       Clearwire   5.620000e+09
21898                       Time Warner       Clearwire   5.620000e+09
3653                        BrightHouse       Clearwire   4.700000e+09
9138                             Google       Clearwire   3.200000e+09
6607               Eagle River Holdings       Clearwire   2.420000e+09
9054                      Goldman Sachs        Facebook   1.500000e+09
2695                   Battery Ventures         Groupon   1.085000e+09
8577                        GI Partners  Wave Broadband   1.050000e+09
16062         Oak Hill Capital Partners  Wave Broadband   1.050000e+09
9133                             Google             AOL   1.000000e+09
1519                Andreessen Horowitz         Groupon   9.500000e+08
7421                  Fidelity Ventures    

In [14]:
#Which funding round was the most popular? Which was the least popular?
sql_query = """
select funding_round_type, sum(raised_amount_usd) as raised_amount 
from crunch 
where raised_amount_usd is not null 
group by funding_round_type;
"""
funding_round_type = pd.read_sql(sql_query, conn)
print(f'the top funding round type :\n{funding_round_type.nlargest(1, columns="raised_amount", keep="all")}')
print(f'the top funding round type :\n{funding_round_type.nsmallest(1, columns="raised_amount", keep="all")}')


the top funding round type :
  funding_round_type  raised_amount
7          series-c+   2.162039e+11
the top funding round type :
  funding_round_type  raised_amount
1       crowdfunding      6491500.0
